In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.0.0-beta1


In [2]:
def make_toy_dataset(num_data):
    X = np.random.randn(num_data, 3)
    y = 3 * X[:, 0] - 2 * X[:, 1]**3 + 2 * X[:, 2]**2 + 0.5 * np.random.randn(num_data)
    y = y[:, np.newaxis]
    return X, y

In [3]:
def build_model():
    model = keras.models.Sequential([
        keras.layers.Dense(10, activation="relu", input_dim=3),
        keras.layers.Dense(10, activation="relu"),
        keras.layers.Dense(1)
    ])

    return model

In [4]:
N_train = 1000
N_test = 200

num_epochs = 25
learning_rate = 0.01
batch_size = 64

In [5]:
# get toy datasets
X_train, y_train = make_toy_dataset(N_train)
X_test, y_test = make_toy_dataset(N_test)

# make datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=512).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(100)

In [6]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


logdir = os.path.join("logs", "manual_logging")
train_logdir = os.path.join(logdir, "train")
test_logdir = os.path.join(logdir, "test")

# create file writer
train_summary_writer = tf.summary.create_file_writer(train_logdir)
test_summary_writer = tf.summary.create_file_writer(test_logdir)


for epoch in range(1, num_epochs+1):
    
    # train step 
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss = loss_fn(y_batch, y_pred)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)
    
    
    # save train metrics
    with train_summary_writer.as_default():
        tf.summary.scalar("loss", train_loss.result(), step=epoch)
        tf.summary.scalar("mae", train_mae_metric.result(), step=epoch)
    
    train_loss.reset_states()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    for x_batch, y_batch in test_dataset:
        y_pred = model(x_batch)
        loss = loss_fn(y_batch, y_pred)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)
    
    
    # save metrics of validation set
    with test_summary_writer.as_default():
        tf.summary.scalar("loss", test_loss.result(), step=epoch)
        tf.summary.scalar("mae", test_mae_metric.result(), step=epoch)
    
    
    test_loss.reset_states()
    test_mae_metric.reset_states()